In [1]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

d = {
    'user_id': [125,23,389,44,505,661,101,690], 
    'user_birthdate': ['01/05/1980','25/08/1975','01/03/1991','10/06/1984',
                       '05/01/1988','09/12/1996','23/07/1975','03/07/1995'],
    'user_name': ['JoHn|','Mi ke','Cha rLi3','Jeff','P4trick','Ma.tt','jAmes', 'Lucas'], 
    'user_money': ['$1,200','$2,555','$754','$980','$3,100','$2,955','$8,500', '$4200']
    }

df = pd.DataFrame(data=d)

In [2]:
store_patterns = [('JoHn|', 'John'), ('Mi ke','Mike'), ('Cha rLi3', 'Charlie'), 
                  ('P4trick','Patrick'), ('Ma.tt', 'Matt'), ('jAmes', 'James')]

def generalize(ser, match_name, default=None, regex=False, case=False):
    seen = None
    for match, name in match_name:
        mask = ser.str.contains(match, case=case, regex=regex)
        if seen is None:
            seen = mask
        else:
            seen |= mask
        ser = ser.where(~mask, name)
    if default:
        ser = ser.where(seen, default)
    else:
        ser = ser.where(seen, ser.values)
        
    return ser


In [3]:
df['user_birthdate'] = df['user_birthdate'].astype('datetime64[ns]')
df['user_name'] = generalize(df['user_name'], store_patterns)
df['user_money'] = (df['user_money']
    .str.findall(r'\d+')
    .str.join('')
    .astype(int))
df = (df
    .query('user_money > 3000' and 'user_birthdate > 1985')
    .sort_values(by='user_money', ascending=False)
    .reset_index(drop=True))

df.head()

,user_id,user_birthdate,user_name,user_money
0,690,1995-03-07,Lucas,4200
1,505,1988-05-01,Patrick,3100
2,661,1996-09-12,Matt,2955
3,389,1991-01-03,Charlie,754
